In [ ]:
# python -m venv genai
# 

In [ ]:
# cd "C:\Users\Lenovo\Desktop\Gen AI"
# genai\Scripts\activate
# pip install -r requirement.txt


# git rm --cached .env

In [ ]:
from dotenv import load_dotenv
import os

from PyPDF2 import PdfReader
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
# If using ConversationalRetrievalChain (legacy):
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferMemory

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Option 1: set the env variable before running your script
# os.environ["GOOGLE_API_KEY"] = "your_gemini_api_key"

# Option 2: pass the API key explicitly
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", google_api_key=api_key)

response = llm.invoke("Explain prompt engineering in one sentence.")
print(response.content)


Prompt engineering is the art and science of crafting clear, concise, and effective instructions to guide AI models towards generating desired outputs.


In [12]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)
    chunks = text_splitter.split_text(text)
    return chunks

get_text_chunks(" Hi     ,there how are you")

['Hi     ,there how are you']

In [20]:
# Sentence Transformer

In [25]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example chunks of text
text_chunks = [
    "First chunk of text to embed.",
    "Second chunk of text, maybe a paragraph.",
    "More text chunks can be added here.",
    "Boiler Tubes are very rare material that withstand lot of pressure",
    "Which is better? Watertube or Firetube boiler?"
]

# Generate embeddings
embeddings = model.encode(text_chunks)

# # Print the embeddings
# for i, embedding in enumerate(embeddings):
#     print(f"Embedding {i}:", embedding)
#     print(f"Vector dimension: {len(embedding)}")

# Save embeddings and texts locally as .npy and .txt files
np.save('embeddings.npy', embeddings)

with open('text_chunks.txt', 'w', encoding='utf-8') as f:
    for chunk in text_chunks:
        f.write(chunk.replace('\n', ' ') + '\n')


In [19]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load embeddings and texts
embeddings = np.load('embeddings.npy')

with open('text_chunks.txt', 'r', encoding='utf-8') as f:
    text_chunks = [line.strip() for line in f]

# Example query
query = "What can you tell me about the embeddings"
query_embedding = model.encode([query])

# Calculate cosine similarities and find best match
similarities = cosine_similarity(query_embedding, embeddings)
best_match_idx = similarities.argmax()

print("Best matching chunk:", text_chunks[best_match_idx])


Best matching chunk: First chunk of text to embed.


In [21]:
# FAISS

In [26]:
import faiss
import numpy as np

# Load model & generate embeddings for text_chunks (as before)
# embeddings = model.encode(text_chunks)

# Convert to float32 for FAISS
embedding_vectors = np.array(embeddings).astype('float32')

# Create FAISS index
dimension = embedding_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embedding_vectors)

# Save index to file for future use
faiss.write_index(index, 'faiss_index.bin')

# To save text_chunks for lookup
with open('text_chunks.txt', 'w', encoding='utf-8') as f:
    for chunk in text_chunks:
        f.write(chunk.replace('\n', ' ') + '\n')


In [27]:
import faiss
import numpy as np

# Load FAISS index and text chunks
index = faiss.read_index('faiss_index.bin')

with open('text_chunks.txt', 'r', encoding='utf-8') as f:
    text_chunks = [line.strip() for line in f]

# Encode query
query = "What can you tell me about the boiler?"
query_vector = model.encode([query]).astype('float32')

# Search top 1 nearest neighbor
distances, indices = index.search(query_vector, k=1)

best_idx = indices[0][0]
print(f"Best matching chunk: {text_chunks[best_idx]}, Distance: {distances[0][0]}")


Best matching chunk: Which is better? Watertube or Firetube boiler?, Distance: 0.8428683280944824


In [29]:
import os
import numpy as np
from PyPDF2 import PdfReader
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
# If using ConversationalRetrievalChain (legacy):
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", google_api_key = GOOGLE_API_KEY)

template = '''
        You are an expert in answering question about the boiler tube leakage.
        here are some relevant information: {relevant_Information}
        Here is the question to answer: {question}
    '''

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | llm

result = chain.invoke({"relevant_Information": [], "question" : "What are different types of boiler tube leakages?"})

print(result)

content='Based on the information provided (which is currently empty, but I\'ll assume general knowledge about boiler tube leakages), here are the different types of boiler tube leakages:\n\nBoiler tube leakages can be broadly categorized by the **cause of failure** and the **location of the failure**.\n\n**1. Leakages by Cause of Failure:**\n\n*   **Erosion:** This occurs when abrasive particles in the flue gas (like ash, coke, or soot) repeatedly impact the tube surface. Over time, this wears away the tube material, thinning it and eventually leading to a hole.\n    *   **Common locations:** Areas with high gas velocity, such as at the back pass, economizer, or superheater sections.\n*   **Corrosion:** This is the degradation of the tube material due to chemical reactions.\n    *   **Water-side Corrosion:** This can be caused by:\n        *   **Oxygen Pitting:** Dissolved oxygen in boiler water can create localized pits that deepen and eventually perforate the tube.\n        *   **Ca

In [ ]:
# Modern way of doing things

In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Option 1: set the env variable before running your script
# os.environ["GOOGLE_API_KEY"] = "your_gemini_api_key"

# Option 2: pass the API key explicitly
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", google_api_key=api_key)

response = llm.invoke("Explain prompt engineering in one sentence.")
print(response.content)

Prompt engineering is the art and science of crafting effective instructions and queries for AI models to elicit desired responses.


In [ ]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Universe."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

pipeline = prompt_template | llm

from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]


from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

pipeline_with_history.invoke(
    {"query": "Hi, my name is Pavan"},
    config={"session_id": "id_123"}
)

pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)